#  Coroutines

    - pep-0342
    - generators are data producers
    - coroutines are data consumers

    - Coroutines consume values using a (yield)


In [3]:
def hello():
    print('helo world')


result = hello()
print(f" {type(hello) =} {type(result)} {result}") 

helo world
 type(hello) =<class 'function'> <class 'NoneType'> None


In [4]:
def hello():
    return 'helo world'


result = hello()
print(f" {type(hello) =} {type(result)} {result}") 

 type(hello) =<class 'function'> <class 'str'> helo world


In [5]:
def hello():
    yield "helo world"   # generator


result = hello()
print(f" {type(hello) =} {type(result)} {result}") 

 type(hello) =<class 'function'> <class 'generator'> <generator object hello at 0x752e6824b320>


In [6]:
next(result)

'helo world'

In [7]:
def hello():  # coroutine
    value = yield "helo world"
    yield value


result = hello()
print(f" {type(hello) =} {type(result)} {result}")

 type(hello) =<class 'function'> <class 'generator'> <generator object hello at 0x752e682a9240>


In [8]:
# NOTE: Coroutine is special generator

In [9]:
next(result)

'helo world'

In [10]:
def hello():  # coroutine
    value = yield 123123
    # yield value
    yield f"Passed value is {value}"


result = hello()
print(f"{type(result)} {result}")

<class 'generator'> <generator object hello at 0x752e682a9180>


In [11]:
# All coroutines must be "primed" by first calling next() (or send(None))
print(f"{next(result) = }")  # 123123

next(result) = 123123


In [12]:
print(f'{result.send("world") = }')

result.send("world") = 'Passed value is world'


In [14]:
try:
    print(f"{result.send('Python')  =}")
except StopIteration:
    print("No more yields to send !!!")

No more yields to send !!!


### Another example

In [15]:
def my_coroutine(num):
    value = yield "default string"           # default string
    yield value                              # first
    yield value                              # first -- second
    value = yield "new string"               # new string
    yield value                              # fourth

In [16]:
# Step 1: To call
c = my_coroutine(4)
print(f"{type(c)    = }")  # <class 'generator'>
print(f"{c          = }")  # <generator object my_coroutine at 0x0000022D8B349A10>

# Step 2: to prime the generator to become coroutine
print(f"{next(c) =}")  # 'default string'

type(c)    = <class 'generator'>
c          = <generator object my_coroutine at 0x752e682aa200>
next(c) ='default string'


In [17]:
# Step 3: Sending values to coroutine
print(f"{c.send('first')  =}")  # 'first'
print(f"{c.send('second') =}")  # 'first'
print(f"{c.send('third')  =}")  # 'new string'

c.send('first')  ='first'
c.send('second') ='first'
c.send('third')  ='new string'


In [18]:
print(f"{c.send('fourth')  =}")  # 'fourth'

c.send('fourth')  ='fourth'


In [19]:
print(f"{c.send('fifth')  =}") 

StopIteration: 

In [20]:
# To close the coroutine
c.close()

In [21]:
print(f"{c.send('fifth')  =}") 

StopIteration: 

####  Infinite reception

In [22]:

def my_coroutine():
    while True:
        received = yield 1234  # values will be received here
        print(f"Received :{received}")


# Step 1: creating the generator
it = my_coroutine()

# Step 2: Prime the coroutine
print(f"{next(it) =}")  # 1234

# Step 3: sending values to coroutine
it.send("First")
it.send("Second")
it.send("third")

for i in range(9):
    it.send(i)

# Step 4: close the coroutine
it.close()

try:
    it.send("fourth")
except StopIteration:
    print("coroutine is closed. Cant send any value")

next(it) =1234
Received :First
Received :Second
Received :third
Received :0
Received :1
Received :2
Received :3
Received :4
Received :5
Received :6
Received :7
Received :8
coroutine is closed. Cant send any value


### finite reception

In [23]:

def my_coroutine(count=5):
    for _ in range(count):
        received = yield 1234  # values will be received here
        print(f"Received :{received}")


# Step 1: creating the generator
it = my_coroutine(5)

# Step 2: Prime the coroutine
print(f"{next(it) =}")

# Step 3: sending values to coroutine
it.send("First")
it.send("Second")
it.send("third")

for i in range(9):
    it.send(i)

# Step 4: close the coroutine
it.close()

try:
    it.send("fourth")
except StopIteration:
    print("coroutine is closed. Cant send any value")

next(it) =1234
Received :First
Received :Second
Received :third
Received :0
Received :1


StopIteration: 

### chaining the coroutines

In [24]:

def producer(sentence, next_coroutine):
    """
    Producer which just split strings and
    feed it to pattern_filter coroutine
    """
    tokens = sentence.split(" ")
    for token in tokens:
        next_coroutine.send(token)
    next_coroutine.close()


def pattern_filter(pattern="ing", next_coroutine=None):
    """
    Search for pattern in received token
    and if pattern got matched, send it to
    print_token() coroutine for printing
    """
    print("Searching for {}".format(pattern))
    try:
        while True:
            token = yield
            if pattern in token:
                next_coroutine.send(token)
    except GeneratorExit:
        print("Done with filtering!!!")


def print_token():
    """
    Act as a sink, simply print the
    received tokens
    """
    print("I'm sink, i'll print tokens")
    try:
        while True:
            token = yield
            print(token)
    except GeneratorExit:
        print("Done with printing!")


if __name__ == "__main__":
    pt = print_token()
    next(pt)

    pf = pattern_filter(next_coroutine=pt)
    next(pf)

    sentence = "Bob is running behind a fast moving car"
    producer(sentence, pf)

I'm sink, i'll print tokens
Searching for ing
running
moving
Done with filtering!!!


### asyncio

In [25]:
import asyncio

print(dir(asyncio))

['ALL_COMPLETED', 'AbstractChildWatcher', 'AbstractEventLoop', 'AbstractEventLoopPolicy', 'AbstractServer', 'Barrier', 'BaseEventLoop', 'BaseProtocol', 'BaseTransport', 'BoundedSemaphore', 'BrokenBarrierError', 'BufferedProtocol', 'CancelledError', 'Condition', 'DatagramProtocol', 'DatagramTransport', 'DefaultEventLoopPolicy', 'Event', 'FIRST_COMPLETED', 'FIRST_EXCEPTION', 'FastChildWatcher', 'Future', 'Handle', 'IncompleteReadError', 'InvalidStateError', 'LifoQueue', 'LimitOverrunError', 'Lock', 'MultiLoopChildWatcher', 'PidfdChildWatcher', 'PriorityQueue', 'Protocol', 'Queue', 'QueueEmpty', 'QueueFull', 'ReadTransport', 'Runner', 'SafeChildWatcher', 'SelectorEventLoop', 'Semaphore', 'SendfileNotAvailableError', 'Server', 'StreamReader', 'StreamReaderProtocol', 'StreamWriter', 'SubprocessProtocol', 'SubprocessTransport', 'Task', 'TaskGroup', 'ThreadedChildWatcher', 'Timeout', 'TimeoutError', 'TimerHandle', 'Transport', 'WriteTransport', '__all__', '__builtins__', '__cached__', '__doc_

In [26]:
# blocking execution- traditional - python,java,c, c++
# statement1 - 5sec
# statement2 - 5sec
# statement3 - 5sec
# 5 + 5 + 5 = 15 sec

# non-blocking execution- node.js. asyncio, goroutines
# statement1 - 5sec
# statement2 - 5sec
# statement3 - 5sec
# 5 sec

In [ ]:
@asyncio.coroutine
def old_style_coroutine():
    print("old_style_coroutine - start")
    yield from asyncio.sleep(1)
    print("old_style_coroutine - end")


# osc = old_style_coroutine()
# print(f'{osc = }')
# print(f'{list(osc) = }')

asyncio.run(old_style_coroutine())



RuntimeError: asyncio.run() cannot be called from a running event loop

In [30]:
# async, await -- new keywords in python 3.7
async def main():
    print("main - start")
    await old_style_coroutine()
    print("main - end")


asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop